In [ ]:
!pip install requests
!pip install scikit-image
!pip install scikit-learn

import sys
assert sys.version_info >= (3, 7)

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from util_func import *

if not cv.useOptimized():
    cv.setUseOptimized(True)
    
cv.useOptimized()



In [ ]:
def k_means_clust(img,K,max_iter=10,eps=1.0,attempts=10,flag=cv.KMEANS_PP_CENTERS):
    """Args:
    img : 3-channel color image
    k: user-defined number of cluster 
    by def, max_iter = 10 , eps = 1, repetitions = 10 , kmeans++
    """
    #reshape and cast
    img = img.reshape((-1,3))
    img = np.float32(img)
    criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, max_iter,eps)
    return cv.kmeans(img,K,None,criteria,attempts,flag)

In [ ]:
def calcGST(inputIMG, w):
    img = np.float32(inputIMG)
    
    # Gradient structure tensor components
    if method_edge=="scharr":
        imgDiffX = cv.Scharr(img, cv.CV_32F, 1, 0)
        imgDiffY = cv.Scharr(img, cv.CV_32F, 0, 1)
    elif method_edge =="":
        imgDiffXY = cv.multiply(imgDiffX, imgDiffY)
        imgDiffXX = cv.multiply(imgDiffX, imgDiffX)
        imgDiffYY = cv.multiply(imgDiffY, imgDiffY)
    
    J11 = cv.boxFilter(imgDiffXX, cv.CV_32F, (w, w))
    J22 = cv.boxFilter(imgDiffYY, cv.CV_32F, (w, w))
    J12 = cv.boxFilter(imgDiffXY, cv.CV_32F, (w, w))
    
    # eigenvalue
    tmp1 = J11+J22
    tmp2 = J11-J22
    tmp2 = cv.multiply(tmp2, tmp2)
    tmp3 = cv.multiply(J12, J12)
    tmp4 = np.sqrt(tmp2 + 4.0*tmp3)
    
    lambda1 = 0.5*(tmp1+tmp4)
    lambda2 = 0.5*(tmp1 - tmp4)
    
    # coherency
    imgCoherencyOut = cv.divide(lambda1-lambda2, lambda1+lambda2)
    
    # orientation calculation
    imgOrientationOut = cv.phase(J22-J11, 2.0*J12, angleInDegrees = True)
    imgOrientationOut = 0.5*imgOrientationOut
    
    return imgCoherencyOut, imgOrientationOut 

In [ ]:
from sklearn.cluster import KMeans

# Load the image
image = cv.imread("images/zebra.jfif")
image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)

# Reshape the image to be a list of pixels
pixels = image_rgb.reshape(-1, 3)

# Define the range of K values to consider
k_values = range(2, 11)

# Lists to store WCSS values for each color space
wcss_bgr = []
wcss_hsv = []
wcss_lab = []

for K in k_values:
    # Perform K-means clustering using your provided function
    _, labels, _ = k_means_clust(pixels, K)
    
    # Calculate WCSS for BGR color space
    wcss_bgr.append(np.sum(np.square(pixels - labels)))

    # Convert RGB to HSV
    hsv_image = cv.cvtColor(image_rgb, cv.COLOR_RGB2HSV)
    hsv_pixels = hsv_image.reshape(-1, 3)
    
    # Perform K-means clustering using your provided function for HSV color space
    _, labels, _ = k_means_clust(hsv_pixels, K)
    
    # Calculate WCSS for HSV color space
    wcss_hsv.append(np.sum(np.square(hsv_pixels - labels)))

    # Convert RGB to LAB
    lab_image = cv.cvtColor(image_rgb, cv.COLOR_RGB2LAB)
    lab_pixels = lab_image.reshape(-1, 3)
    
    # Perform K-means clustering using your provided function for LAB color space
    _, labels, _ = k_means_clust(lab_pixels, K)
    
    # Calculate WCSS for LAB color space
    wcss_lab.append(np.sum(np.square(lab_pixels - labels)))

# Plot the WCSS vs number of clusters for each color space
plt.figure(figsize=(10, 6))
plt.plot(k_values, wcss_bgr, marker='o', label='BGR')
plt.plot(k_values, wcss_hsv, marker='o', label='HSV')
plt.plot(k_values, wcss_lab, marker='o', label='LAB')
plt.title('Within-Cluster Sum of Squares vs Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.legend()
plt.grid(True)
plt.show()